In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os

os.environ["SPOTIPY_CLIENT_ID"] = "28ba49dab9c6456c95496dd9f969c2f3"
os.environ["SPOTIPY_CLIENT_SECRET"] = "8c89631fcbbc40ec8f0f678236db3a0c"
os.environ["SPOTIPY_REDIRECT_URI"] = 'http://localhost:5000'

In [3]:
scope = 'user-top-read user-library-modify'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_top_tracks(time_range='short_term')

In [71]:
import itertools

top_tracks = sp.current_user_top_tracks(time_range='medium_term', limit=30)
feature_list = [0, 0, 0, 0, 0]
genre_list = []
for item in top_tracks['items']:
    url = item['external_urls']['spotify']
    artist_id = item['album']['artists'][0]['id']
    genres = sp.artists([artist_id])['artists'][0]['genres']
    features = list(sp.audio_features([url])[0].items())

    dance = features[0][1]
    energy = features[1][1]
    speechiness = features[5][1]
    acousticness = features[6][1]
    lively = features[8][1]
    
    max_list = [dance,energy,speechiness,acousticness,lively]
    max_val = max(max_list)
    max_index = max_list.index(max_val)

    for genre in genres:
        genre_list.append(genre)
    feature_list[max_index] += 1
# top_tracks

In [72]:
from collections import Counter 

counter = Counter(genre_list).most_common()

In [73]:
# store = []
# for features in feature_list:
#     for feature in features:
#         store = [feature]
for val in feature_list:
    print(round(val/40*100, 1))

62.5
12.5
0.0
0.0
0.0


In [28]:
# Featured Playlists

# sp.category_playlists('toplists', country='gb')

In [10]:
# # Spotify Recommendations

# info_list = []
# results = sp.current_user_top_artists(time_range='short_term', limit=5)
# artist_list = []
# for artist_item in results['items']:
#     seed_id = artist_item['id']
#     recommendations = sp.recommendations(seed_artists=[seed_id], limit=3)
#     for item in recommendations['tracks']:
#         print(item['id'])
#         break
#         # track = item['name']
#         # image = item['album']['images'][1]['url']
#         # artist = item['album']['artists'][0]['name']
#         # artist_id = item['album']['artists'][0]['id']
#         # album = item['album']['name']
#         # info_list.append([track, image, artist, artist_id])

0SGkqnVQo9KPytSri1H6cF
43PuMrRfbyyuz4QpZ3oAwN
5wJL4o7k8m02m6ZM9KT0ir
0jqBo5RYn008f4ZY8kPewW
5z6f5HNjkDhzdouZp5dYtx


In [5]:
info_list

[['On The Way',
  'https://i.scdn.co/image/ab67616d00001e02625b914e19234d4ab74ffc1e',
  'Russ',
  '1z7b1Pr1rSlvWRzsW3HOrS'],
 ['I Know',
  'https://i.scdn.co/image/ab67616d00001e0250192d5f728fea13fb3af203',
  'Big Sean',
  '0c173mlxpT3dSFRgMO8XPh'],
 ['BIG BANK (feat. 2 Chainz, Big Sean, Nicki Minaj)',
  'https://i.scdn.co/image/ab67616d00001e02f43405f32dfe1c07777c06af',
  'YG',
  '0A0FS04o6zMoto8OKPsDwY'],
 ['PRBLMS',
  'https://i.scdn.co/image/ab67616d00001e02e4f13cd87b3b1ead6735a520',
  '6LACK',
  '4IVAbR2w4JJNJDDRFP3E83'],
 ['1985 - Intro to “The Fall Off”',
  'https://i.scdn.co/image/ab67616d00001e02cf0f0affd0138a7442f13706',
  'J. Cole',
  '6l3HvQ5sa6mXTsMTB19rO5'],
 ['Aston Martin Music',
  'https://i.scdn.co/image/ab67616d00001e020fbb5046ac2211a4aab5ecfa',
  'Rick Ross',
  '1sBkRIssrMs1AbVkOJbc7a'],
 ['Forbes',
  'https://i.scdn.co/image/ab67616d00001e02251a350fdbfad75c0c0c9038',
  'G-Eazy',
  '02kJSzxNuaWGqwubyUba0Z'],
 ['untitled',
  'https://i.scdn.co/image/ab67616d00001e022

In [16]:
for item in results['items']:
    print(item['id'])
    break


3JKgcAa7r07ocVWcV8bS0H


In [19]:
track_res = sp.track('3JKgcAa7r07ocVWcV8bS0H')
# artist_id = sp.artists([track_res['artists'][0]['id']])
track_res['album']['images'][0]['url']



'https://i.scdn.co/image/ab67616d0000b273339f780dfdbc38558ea0761d'

In [23]:
artist = track_res['album']['artists'][0]['name']
name = track_res['name']
popularity = track_res['popularity']

url = track_res['external_urls']['spotify']

url
# list(sp.audio_features(['2AlYncTpVHKwHb55F9lF6O'])[0].items())

'https://open.spotify.com/track/3JKgcAa7r07ocVWcV8bS0H'

In [5]:
track_info = []
for track in results['items']:
    print(track['album']['artists'][0]['id'])
    break
    # track_info.append([track['name'], track['album']['artists'][0]['name'], track['album']['images'][1]['url']])
# track_info

KeyError: 'album'

In [ ]:
results.keys()
track_list = []
i = 0
for track in results['items']:
    track_list.append(track['external_urls']['spotify'])
    if i > 2:
        break
    i += 1

In [38]:
search_result = sp.search('if i lead')
artist_id = search_result['tracks']['items'][0]['album']['artists'][0]['id']
image = search_result['tracks']['items'][0]['album']['images'][1]['url']
artist_name = search_result['tracks']['items'][0]['album']['artists'][0]['name']
print(artist_id)
print(image)
print(artist_name)


27CC3tpq7WQR25M03jKTZm
https://i.scdn.co/image/ab67616d00001e023003515eb745f18d09b4206f
Kiltro


In [31]:
artist_id

'0xOeVMOz2fVg5BJY3N6akT'

In [ ]:
top_artists  = sp.current_user_top_artists()
artist_dict = {}
i = 0
for artist in top_artists['items']: 
    artist_dict[artist['name']] = [artist['images'][1]['url'], artist['followers']['total'], artist['genres'], artist['external_urls']['spotify'] ]
    if i > 2:
        break
    i += 1
    print(artist['id'])
# artist_dict


6l3HvQ5sa6mXTsMTB19rO5
7Ln80lUS6He07XvHI8qqHH
4Z8W4fKeB5YxbusRsdQVPb


In [ ]:
search_result = sp.artists(["6l3HvQ5sa6mXTsMTB19rO5"])
search_result['artists'][0]['id']
albums = sp.artist_albums('6l3HvQ5sa6mXTsMTB19rO5', album_type='album')

In [11]:
albums['items']
album_info = {}
for item in albums['items']: 
    if item['name'] not in album_info:
        album_info[item['name']] = [item['release_date'], item['images'][0]['url']]
list(album_info.items())

[('The Off-Season',
  ['2021-05-14',
   'https://i.scdn.co/image/ab67616d0000b27310e6745bb2f179dd3616b85f']),
 ("Revenge Of The Dreamers III: Director's Cut",
  ['2020-01-16',
   'https://i.scdn.co/image/ab67616d0000b273a145ed96ce61bf4201d619c3']),
 ('Revenge Of The Dreamers III',
  ['2019-07-05',
   'https://i.scdn.co/image/ab67616d0000b273c0ae820e3b64854de4642101']),
 ('KOD',
  ['2018-04-20',
   'https://i.scdn.co/image/ab67616d0000b273cf0f0affd0138a7442f13706']),
 ('4 Your Eyez Only',
  ['2016-12-09',
   'https://i.scdn.co/image/ab67616d0000b273f4ca75192df162f78a24023e']),
 ('Forest Hills Drive: Live from Fayetteville, NC',
  ['2016-01-28',
   'https://i.scdn.co/image/ab67616d0000b2737617e402e6b7fef18f4fa7c9']),
 ('2014 Forest Hills Drive',
  ['2014-12-09',
   'https://i.scdn.co/image/ab67616d0000b2736aca031ccc27d2e4dd829d14']),
 ('The Blow Up',
  ['2014-11-09',
   'https://i.scdn.co/image/ab67616d0000b273bcdfc0c2ca343f704cc36934']),
 ('Born Sinner (Deluxe Version)',
  ['2013-06-18'

In [20]:
artist_top_tracks_dict = {}
url_list = []
for artist in artist_dict.values():
    print(artist[3])
    # top = sp.artist_top_tracks("6l3HvQ5sa6mXTsMTB19rO5")['tracks']
    top = sp.artist_top_tracks(artist[3])['tracks']
    for track in top:
        # print(track['artists']['id'])
        url = (track['external_urls']['spotify'])
        artist_top_tracks_dict[track['name']] = url
    break
list(sp.audio_features([url])[0].items())

https://open.spotify.com/artist/6l3HvQ5sa6mXTsMTB19rO5


[('danceability', 0.784),
 ('energy', 0.474),
 ('key', 8),
 ('loudness', -6.139),
 ('mode', 1),
 ('speechiness', 0.0524),
 ('acousticness', 0.333),
 ('instrumentalness', 0),
 ('liveness', 0.278),
 ('valence', 0.64),
 ('tempo', 128.012),
 ('type', 'audio_features'),
 ('id', '15qKrPi4Kg0z7g9HlwWyeb'),
 ('uri', 'spotify:track:15qKrPi4Kg0z7g9HlwWyeb'),
 ('track_href', 'https://api.spotify.com/v1/tracks/15qKrPi4Kg0z7g9HlwWyeb'),
 ('analysis_url',
  'https://api.spotify.com/v1/audio-analysis/15qKrPi4Kg0z7g9HlwWyeb'),
 ('duration_ms', 204434),
 ('time_signature', 4)]

In [ ]:
sp.curr

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
artist_df = pd.read_csv('../../data/artists.csv')
tracks_df = pd.read_csv('../../data/tracks.csv')
with open("../../data/dict_artists.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [3]:
artist_df.drop(['id'], axis=1, inplace=True)

In [4]:
artist_df = artist_df[artist_df['genres'] != '[]']
artist_df = artist_df.sort_values(by=['popularity'], ascending=False)
l = len(artist_df)/15
artist_df = artist_df[:round(l)]
artist_df = artist_df.reset_index(drop=True)

In [5]:
artist_df

,followers,genres,name,popularity
0,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",Justin Bieber,100
1,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",Drake,98
2,32244734.0,"['latin', 'reggaeton', 'trap latino']",Bad Bunny,98
3,38869193.0,"['pop', 'post-teen pop']",Taylor Swift,98
4,16996777.0,"['chicago rap', 'melodic rap']",Juice WRLD,96
...,...,...,...,...
19903,2394.0,['russian edm'],Denis First,50
19904,1028316.0,"['sertanejo', 'sertanejo pop', 'sertanejo univ...",Jefferson Moraes,50
19905,33168.0,"['danish hip hop', 'danish pop']",Bro,50
19906,15823.0,"['bass house', 'deep groove house', 'electro h...",Marten Hørger,50


In [ ]:
# orig_model = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
# tfidf_matrix_orig = orig_model.fit_transform(artist_df['genres'])

In [6]:

model = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words=None)
tfidf_matrix = model.fit_transform(artist_df['genres'])

In [7]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 

In [8]:
indices = pd.Series(artist_df.index, index=artist_df['name'])

In [9]:
results = {}
for idx, row in artist_df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], artist_df['name'][i]) for i in similar_indices] 
    results[row['name']] = similar_items[1:]

In [10]:
def _recommend(item_id, num):
    recs = results[item_id][:num]   
    preds = {}
    for pair in recs:
        preds[pair[1]] = pair[0]
    return preds

In [14]:
_recommend('Ed Sheeran', 5)

{'Lewis Capaldi': 1.0000000000000002,
 'Ed Sheeran': 1.0000000000000002,
 'Dua Lipa': 0.7212451083169278,
 'HONNE': 0.6876996711411917,
 'Louisa': 0.6867634867587893}

In [54]:
def _recommend_multiple(artists, num=10):
    dict_similar = {}
    for artist, weight in artists.items():
        dict_similar[artist] = _recommend(artist, num)
    artists_all = []
    for artist, similar_artists in dict_similar.items():
        artists_all.append(list(similar_artists.keys()))
    artists_unique = np.unique(artists_all).tolist()
    artists_dict = {artist: 0 for artist in artists_unique}
    for artist, similar_artists in dict_similar.items():
        for similar_artist, score in similar_artists.items():
            artists_dict[similar_artist] += artists[artist] * score
    return list({k: v for k, v in sorted(artists_dict.items(), key=lambda item: item[1], reverse=True) if k not in artists}.keys())[0:num]

In [ ]:
_recommend_multiple({"Drake": 10, "Queen": 8}, 5)

In [15]:
import pickle 
with open('cos_sim_results','wb') as f: pickle.dump(results, f)

In [4]:
with open('../pages/pickle/cos_sim_results', 'rb') as f:
    results = pickle.load(f)

dict